In [17]:
import ee
import geemap

# Authenticate to Earth Engine
try:
  ee.Initialize()
except Exception as e:
  ee.Authenticate()
  ee.Initialize(project='ee-ana-zonia')

from utils import export_image
from utils import map_image

roi = ee.FeatureCollection("projects/ee-ana-zonia/assets/br_biomes").geometry().dissolve()

first_year = 1985
last_year = 2020

# Load images from MapBiomas Collection 8 for Land Use Land Cover and Burned Area
lulc = ee.Image("projects/mapbiomas-workspace/public/collection8/mapbiomas_collection80_integration_v1") \
  .select([f"classification_{year}" for year in range(first_year, last_year+1)]).byte()
fire = ee.Image("projects/mapbiomas-workspace/public/collection7_1/mapbiomas-fire-collection2-annual-burned-coverage-1") \
  .select([f"burned_coverage_{year}" for year in range(first_year, last_year)]).byte()

## Land use and land cover

### Fire
Note that fire has different transform than lulc, so the projection will need to be adjusted when exporting.

In [13]:
# fire has the value of the land use type that burned.
# Transforming into a fire mask:
fire = fire.gt(0)
num_fires = fire.reduce(ee.Reducer.sum()).rename('num_fires').byte()

# get fire frequency data from Mapbiomas - double check it.
# fire_freq = ee.Image("projects/mapbiomas-workspace/public/collection7_1/mapbiomas-fire-collection2-fire-frequency-1").clip(roi)
# fire_freq = fire_freq.select('fire_frequency_1985_2020')

# how many years ago was each fire? #############################
# Get the number of bands
num_bands = fire.bandNames().size()
# Create a sequence of numbers from 1 to num_bands
years_ago = ee.List.sequence(1, num_bands)
years_ago = years_ago.reverse()

# # # Map over the image and set values based on the band index
constant_images = ee.ImageCollection.fromImages(
    years_ago.map(lambda year: ee.Image.constant(year))).toBands()

time_since_all_fires = fire.multiply(constant_images)

old_names = time_since_all_fires.bandNames().getInfo()
new_names = [name.replace('burned_coverage', 'time_since_fire') for name in old_names]
time_since_all_fires = time_since_all_fires.select(old_names).rename(new_names)

# how many years ago was the LAST fire? #############################
last_fire = time_since_all_fires.reduce(ee.Reducer.lastNonNull()).rename('last_fire').byte()

In [15]:
# Define the export parameters
export_params = {
    'image': num_fires,
    'description': 'num_fires',
    'assetId': 'projects/ee-ana-zonia/assets/num_fires',
    'scale': 30,
    'crs': 'EPSG:4326',
    'region': roi
}

# Create the export task
task = ee.batch.Export.image.toAsset(**export_params)

# Start the export task
task.start()

# Define the export parameters
export_params = {
    'image': last_fire,
    'description': 'last_fire_notamaz',
    'assetId': 'projects/ee-ana-zonia/assets/last_fire_notamaz',
    'scale': 30,
    'crs': 'EPSG:4326',
    'region': roi,
    'maxPixels' = 4e10
}

# Create the export task
task = ee.batch.Export.image.toAsset(**export_params)

# Start the export task
task.start()